## LAYERS WITHOUT PARAMETERS

In [1]:
from mxnet import gluon, np, npx
from mxnet.gluon import nn
npx.set_np()

In [2]:
class CenteredLayer(nn.Block):
    def __init__(self, **kwargs):
        super(CenteredLayer, self).__init__(*kwargs)
        
    def forward(self, x):
        return x - x.mean()

In [3]:
layer = CenteredLayer()
layer(np.array([1, 2, 3, 4, 5]))

array([-2., -1.,  0.,  1.,  2.])

In [4]:
net = nn.Sequential()
net.add(nn.Dense(128), CenteredLayer())
net.initialize()

In [5]:
y = net(np.random.uniform(size= (4, 8)))
y.mean()

array(3.783498e-10)

## LAYERS WITH PARAMETERS

In [6]:
params = gluon.ParameterDict()
params.get('param2', shape=(2,3))

Parameter param2 (shape=(2, 3), dtype=<class 'numpy.float32'>)

In [7]:
class MyDense(nn.Block):
    def __init__(self, units, in_units, **kwargs):
        super(MyDense, self).__init__(**kwargs)
        
        self.weight = self.params.get('weight', shape= (in_units, units))
        self.bias = self.params.get('bias', shape=(units,))
        
    def forward(self, x):
        linear = np.dot(x, self.weight.data()) + self.bias.data()
        return npx.relu(linear)
    

In [8]:
dense = MyDense(units=3, in_units=5)
dense.params

mydense0_ (
  Parameter mydense0_weight (shape=(5, 3), dtype=<class 'numpy.float32'>)
  Parameter mydense0_bias (shape=(3,), dtype=<class 'numpy.float32'>)
)

In [9]:
dense.initialize()
dense(np.random.uniform(size= (2,5)))

array([[0.        , 0.01633355, 0.        ],
       [0.        , 0.01581812, 0.        ]])

In [10]:
net = nn.Sequential()
net.add(MyDense(8, in_units=64),
       MyDense(1, in_units=8))
net.initialize()
net(np.random.uniform(size=(2,64)))

array([[0.06508517],
       [0.0615553 ]])